# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 22 2022 8:30AM,242484,10,MSP212432,"Methapharm, Inc.",Released
1,Jul 22 2022 8:30AM,242484,10,MSP212425,"Methapharm, Inc.",Released
2,Jul 21 2022 3:31PM,242485,10,MSP212433,"Methapharm, Inc.",Released
3,Jul 21 2022 3:31PM,242485,10,MSP212436,"Methapharm, Inc.",Released
4,Jul 21 2022 3:31PM,242485,10,MSP212438,"Methapharm, Inc.",Released
5,Jul 21 2022 3:31PM,242485,10,MSP212439,"Methapharm, Inc.",Released
6,Jul 21 2022 3:31PM,242485,10,MSP212440,"Methapharm, Inc.",Released
7,Jul 21 2022 3:31PM,242485,10,MSP212442,"Methapharm, Inc.",Released
8,Jul 21 2022 3:31PM,242485,10,MSP212422,"Methapharm, Inc.",Released
9,Jul 21 2022 3:31PM,242485,10,MSP212423,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,242481,Released,2
40,242482,Released,1
41,242483,Released,1
42,242484,Released,2
43,242485,Released,15


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242481,NaN,2.0
242482,NaN,1.0
242483,NaN,1.0
242484,NaN,2.0
242485,NaN,15.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
242348,53.0,24.0
242375,0.0,1.0
242376,1.0,0.0
242390,0.0,1.0
242391,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
242348,53,24
242375,0,1
242376,1,0
242390,0,1
242391,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,242348,53,24
1,242375,0,1
2,242376,1,0
3,242390,0,1
4,242391,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,242348,53,24
1,242375,,1
2,242376,1,
3,242390,,1
4,242391,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 22 2022 8:30AM,242484,10,"Methapharm, Inc."
2,Jul 21 2022 3:31PM,242485,10,"Methapharm, Inc."
17,Jul 21 2022 3:20PM,242483,10,Emerginnova
18,Jul 21 2022 3:16PM,242482,20,"Exact-Rx, Inc."
19,Jul 21 2022 3:13PM,242481,10,ISDIN Corporation
21,Jul 21 2022 3:12PM,242479,10,ISDIN Corporation
45,Jul 21 2022 3:09PM,242480,19,"AdvaGen Pharma, Ltd"
48,Jul 21 2022 3:03PM,242476,10,ISDIN Corporation
71,Jul 21 2022 2:53PM,242477,10,Emerginnova
72,Jul 21 2022 2:51PM,242473,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 22 2022 8:30AM,242484,10,"Methapharm, Inc.",,2
1,Jul 21 2022 3:31PM,242485,10,"Methapharm, Inc.",,15
2,Jul 21 2022 3:20PM,242483,10,Emerginnova,,1
3,Jul 21 2022 3:16PM,242482,20,"Exact-Rx, Inc.",,1
4,Jul 21 2022 3:13PM,242481,10,ISDIN Corporation,,2
5,Jul 21 2022 3:12PM,242479,10,ISDIN Corporation,,24
6,Jul 21 2022 3:09PM,242480,19,"AdvaGen Pharma, Ltd",1,2
7,Jul 21 2022 3:03PM,242476,10,ISDIN Corporation,,23
8,Jul 21 2022 2:53PM,242477,10,Emerginnova,,1
9,Jul 21 2022 2:51PM,242473,10,ISDIN Corporation,,41


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 8:30AM,242484,10,"Methapharm, Inc.",2,
1,Jul 21 2022 3:31PM,242485,10,"Methapharm, Inc.",15,
2,Jul 21 2022 3:20PM,242483,10,Emerginnova,1,
3,Jul 21 2022 3:16PM,242482,20,"Exact-Rx, Inc.",1,
4,Jul 21 2022 3:13PM,242481,10,ISDIN Corporation,2,
5,Jul 21 2022 3:12PM,242479,10,ISDIN Corporation,24,
6,Jul 21 2022 3:09PM,242480,19,"AdvaGen Pharma, Ltd",2,1
7,Jul 21 2022 3:03PM,242476,10,ISDIN Corporation,23,
8,Jul 21 2022 2:53PM,242477,10,Emerginnova,1,
9,Jul 21 2022 2:51PM,242473,10,ISDIN Corporation,41,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 8:30AM,242484,10,"Methapharm, Inc.",2,
1,Jul 21 2022 3:31PM,242485,10,"Methapharm, Inc.",15,
2,Jul 21 2022 3:20PM,242483,10,Emerginnova,1,
3,Jul 21 2022 3:16PM,242482,20,"Exact-Rx, Inc.",1,
4,Jul 21 2022 3:13PM,242481,10,ISDIN Corporation,2,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 8:30AM,242484,10,"Methapharm, Inc.",2.0,NaN
1,Jul 21 2022 3:31PM,242485,10,"Methapharm, Inc.",15.0,NaN
2,Jul 21 2022 3:20PM,242483,10,Emerginnova,1.0,NaN
3,Jul 21 2022 3:16PM,242482,20,"Exact-Rx, Inc.",1.0,NaN
4,Jul 21 2022 3:13PM,242481,10,ISDIN Corporation,2.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 8:30AM,242484,10,"Methapharm, Inc.",2.0,0.0
1,Jul 21 2022 3:31PM,242485,10,"Methapharm, Inc.",15.0,0.0
2,Jul 21 2022 3:20PM,242483,10,Emerginnova,1.0,0.0
3,Jul 21 2022 3:16PM,242482,20,"Exact-Rx, Inc.",1.0,0.0
4,Jul 21 2022 3:13PM,242481,10,ISDIN Corporation,2.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5334105,139.0,1.0
12,242445,1.0,0.0
16,1212097,9.0,0.0
19,727378,5.0,5.0
20,969719,44.0,53.0
21,1212073,4.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5334105,139.0,1.0
1,12,242445,1.0,0.0
2,16,1212097,9.0,0.0
3,19,727378,5.0,5.0
4,20,969719,44.0,53.0
5,21,1212073,4.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,139.0,1.0
1,12,1.0,0.0
2,16,9.0,0.0
3,19,5.0,5.0
4,20,44.0,53.0
5,21,4.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,139.0
1,12,Released,1.0
2,16,Released,9.0
3,19,Released,5.0
4,20,Released,44.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Executing,1.0,0.0,0.0,5.0,53.0,1.0
Released,139.0,1.0,9.0,5.0,44.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Executing,1.0,0.0,0.0,5.0,53.0,1.0
1,Released,139.0,1.0,9.0,5.0,44.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Executing,1.0,0.0,0.0,5.0,53.0,1.0
1,Released,139.0,1.0,9.0,5.0,44.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()